<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-03-23
-->
[Home](../index.ipynb) / Mikrofon ADMP401
***
# Mikrofon ADMP401

* [Datenblatt ADMP401 Chip](doc/ADMP401.pdf) (Lokale Kopie), Quelle: [sparkfun.com](https://www.sparkfun.com/datasheets/Components/General/ADMP401.pdf)
* [Schlatplan ADMP401 Breakout Board](doc/ADMP401-carrier-schematic.pdf) (Lokale Kopie), Quelle: [pololu.com](https://www.pololu.com/file/0J394/ADMP401-carrier-schematic.pdf)


In [38]:
#%serialconnect --port=COM3 # Windows with more than one COM-Port
%serialconnect # Linux / Windows with one COM-Port

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [57]:
# Usefull class to get elapsed time:
import time

class Timer():
    def __init__(self, name=None):
        self.name = name
#        self.getTime    = time.time_ns   # Python
#        self.TIME_SCALE = 10E9
        self.getTime    = time.ticks_ms # MicroPython
        self.TIME_SCALE = 1000
    def __enter__(self):
        self.start()
        
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.stop()

    def start( self ):
        self.t_start = self.getTime()

    def stop( self ):
        if self.name:
            print('[{}]'.format(self.name), end=" " )
            
        print('Total time: {} s'.format((self.getTime() - self.t_start)/self.TIME_SCALE))

In [39]:
from math import pi, cos

class WindowedRingBuffer :
    def __init__(self,length):
        self.length   = length
        self.iLast    = length
        self.aData    = [0]*length # [i for i in range( length )]

        # Nuttall window, see https://en.wikipedia.org/wiki/Window_function#Hann_and_Hamming_windows
        self.aWindow  = [0.355768-0.487396*cos(2*pi*n/length)+0.144232*cos(4*pi*n/length)-0.012604*cos(6*pi*n/length) for n in range(length)] # Hanning window
                        # Hanning = 0.5 – 0.5*cos(2*pi * n / length )

    def get(self, iIndex):
        return self.aData[ (self.iLast + iIndex) % self.length ]*self.aWindow[ iIndex ]
        
    def add(self,val):
        self.iLast = (self.iLast - 1) % self.length
        self.aData[self.iLast] = val

    def __str__(self) :
        return( "{} {}".format( self.__class__, [ self.get(iIndex) for iIndex in range(self.length) ] ) )

In [40]:
###############################################################################
# ValueSource
#
# Wrapper for getting data.
###############################################################################


class ValueSource :
    def getValue( self ):
        return 0
    
    
    def iter( self, length = -1 ):
        if  length < 0 :
            while True:
                yield self.getValue()
        else:
            for iIndex in range( length ):
                yield self.getValue()
    

#==============================================================================
from math import cos, pi

class MockValueSource( ValueSource ):
    
    def __init__( self, framesPerGet = 1 ):
        self.x = 0
        self.scale = 1000
        self.PI2 = 2*pi
        
    def open( self ):
        return self
    
    def close(self):
        pass

    def getValue( self ):
        self.x+=0.01
        if ( self.x >= self.PI2 ):
            self.x = 0
            
        return int( self.scale*cos( self.x )+self.scale/2*cos( 2*self.x )+self.scale/4*cos( 4*self.x ) )
    

#==============================================================================
# Usage:
#N_SAMPLE = 128
#
#import matplotlib.pyplot as plt
#
#aX = [x for x in range(N_SAMPLE)]
#with WavValueSource( 16 ) as source:
#    aY = [ s for s in source.iter( N_SAMPLE ) ]
#    
## print( aY )
#plt.figure(figsize=(20, 2))
#plt.scatter(aX, aY, s=1, marker='o', color="black" )
#plt.show()
    
    

In [41]:
import math

class FFT:
    TWO_PI = 2 * math.pi
    
    def __init__( self, iSamples = 128 ):
        self.iSamples = iSamples
        
        self.yIn  = [0]*iSamples
        self.xOut = [0]*iSamples
        self.yOut = [0]*iSamples

        self.bufX = WindowedRingBuffer( iSamples )

    def fft( self, bufXIn, direction = 1 ):
        n = bufXIn.length
        factor = self.TWO_PI * direction

        for i in range( n ):
            self.xOut[i] = 0
            self.yOut[i] = 0

            arg = factor * i / n

            for k in range( n ):
                a = k * arg
                cosarg = math.cos(a);
                sinarg = math.sin(a);

                self.xOut[i] += ( bufXIn.get(k) * cosarg - self.yIn[k] * sinarg )
                self.yOut[i] += ( bufXIn.get(k) * sinarg + self.yIn[k] * cosarg )

        return self.yOut

In [60]:
N_SAMPLE = 2**6
print( N_SAMPLE )
bufferX = WindowedRingBuffer( N_SAMPLE )
fft     = FFT( N_SAMPLE )

source = MockValueSource()
source.open()

for iIndex in range( N_SAMPLE -1):
    bufferX.add(source.getValue())

with Timer():
    for i in range( N_SAMPLE ):
        bufferX.add(source.getValue())
        fft.fft( bufferX )


64
...............Total time: 72.955 s


## Sample

In [54]:
from machine import ADC, Pin
import display
display = display.Display()

from time import sleep

pin = Pin(33, Pin.IN, None)

#adc = ADC(0)       # Croduino
adc = ADC( pin ) # HelTec, NodeMCU
adc.atten(ADC.ATTN_0DB)   # [1,1 V ~ 65535]


iMax = 0
iX = 0
while True:
    iValue = adc.read_u16()  # read a raw analog value in the range 0-65535 (= 2^16 - 1)
    display.fill_rect( iX, 0, 1, 64, 0 )
    display.vline( iX, int(64-iValue/5000*54), int(iValue/5000*54) )
    iX = ((iX+1)%128)
    
    #iValue = adc.read()      # Croduino: read a raw analog value in the range 0-511 (= 2^7 - 1)
    
#    display.text( "{:0>6} {:0>6}".format(iValue,iMax), 0, 0 )
    display.show()
    

......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 25, in <module>
KeyboardInterrupt: 
